In [1]:
from function import pdb2pqr
input_path = './data/pdbs'
output_path = './data/pqrs'
pdb2pqr(input_path = input_path, output_path = output_path)

" from function import pdb2pqr\ninput_path = './data/pdbs'\noutput_path = './data/pqrs'\npdb2pqr(input_path = input_path, output_path = output_path) "

In [ ]:
import numpy as np
import trimesh
protein_coords = np.array###!!!!!!!!!!!!!

mesh = trimesh.Trimesh(protein_coords, np.ones((protein_coords.shape[0], 3)))
alpha = 0.4
mesh.alpha_shape(alpha)
surface_area = mesh.area
print("Estimated surface area of the protein:", surface_area)